### Audio classification with a pipeline

Audio classification involves assigning one or more labels to an audio recording based on its content. The labels could correspond to different sound categories, such as music, speech, or noise, or more specific categories like bird song or car engine sounds.

Before diving into details on how the most popular audio transformers work, and before fine-tuning a custom model, let’s see how you can use an off-the-shelf pre-trained model for audio classification with only a few lines of code with 🤗 Transformers.

Let’s go ahead and use the same MINDS-14 dataset that you have explored in the previous unit. If you recall, MINDS-14 contains recordings of people asking an e-banking system questions in several languages and dialects, and has the intent_class for each recording. We can classify the recordings by intent of the call.

Just as before, let’s start by loading the en-AU subset of the data to try out the pipeline, and upsample it to 16kHz sampling rate which is what most speech models require.

In [1]:
from datasets import load_dataset
from datasets import Audio

minds = load_dataset("PolyAI/minds14", name="en-AU", split="train", trust_remote_code=True)
minds = minds.cast_column("audio", Audio(sampling_rate=16_000))

To classify an audio recording into a set of classes, we can use the audio-classification pipeline from 🤗 Transformers. In our case, we need a model that’s been fine-tuned for intent classification, and specifically on the MINDS-14 dataset. 

Luckily for us, the Hub has a model that does just that! Let’s load it by using the pipeline() function:

In [2]:
from transformers import pipeline

classifier = pipeline(
    "audio-classification",
    model="anton-l/xtreme_s_xlsr_300m_minds14",
)

2025-04-03 14:15:06.676996: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

Device set to use cpu


model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

This pipeline expects the audio data as a NumPy array. All the preprocessing of the raw audio data will be conveniently handled for us by the pipeline. Let’s pick an example to try it out:

In [3]:
example = minds[0]

If you recall the structure of the dataset, the raw audio data is stored in a NumPy array under ["audio"]["array"], let’s pass it straight to the classifier:

In [4]:
classifier(example["audio"]["array"])

[{'score': 0.9625310301780701, 'label': 'pay_bill'},
 {'score': 0.02867283672094345, 'label': 'freeze'},
 {'score': 0.0033498068805783987, 'label': 'card_issues'},
 {'score': 0.0020058073569089174, 'label': 'abroad'},
 {'score': 0.0008484343416057527, 'label': 'high_value_payment'},
 {'score': 0.0007367958896793425, 'label': 'direct_debit'},
 {'score': 0.0004057000332977623, 'label': 'latest_transactions'},
 {'score': 0.00033970808726735413, 'label': 'joint_account'},
 {'score': 0.0003312795306555927, 'label': 'address'},
 {'score': 0.0003288653097115457, 'label': 'balance'},
 {'score': 0.00014877533249091357, 'label': 'app_error'},
 {'score': 0.00014772501890547574, 'label': 'atm_limit'},
 {'score': 8.81568921613507e-05, 'label': 'cash_deposit'},
 {'score': 6.512486288556829e-05, 'label': 'business_loan'}]

The model is very confident that the caller intended to learn about paying their bill. Let’s see what the actual label for this example is:

In [5]:
id2label = minds.features["intent_class"].int2str
id2label(example["intent_class"])

'pay_bill'

Hooray! The predicted label was correct! Here we were lucky to find a model that can classify the exact labels that we need. A lot of the times, when dealing with a classification task, a pre-trained model’s set of classes is not exactly the same as the classes you need the model to distinguish. In this case, you can fine-tune a pre-trained model to “calibrate” it to your exact set of class labels. We’ll learn how to do this in the upcoming units. Now, let’s take a look at another very common task in speech processing, automatic speech recognition.